### Dependencies

In [1]:
!pip install matplotlib tensorflow tensorflow_addons tensorflow_datasets imageio

### Setup

In [2]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time
from tensorflow.keras import Model
import tensorflow as tf

from IPython import display

### Dataset

In [3]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  "/home/tony/TO_BE_REMOVED/celeba_data/imgs/",
  seed=123,
  image_size=(64, 64),
  batch_size=256)

Found 202599 files belonging to 1 classes.


In [5]:
# train_images = tf.image.resize(train_images, (32,32)) # if we want to resize 
# train_images = tf.expand_dims(train_images, -1)
# train_images = tf.image.resize(train_images, (32,32))
# print(train_images.shape)
# plt.imshow(train_images[6, :, :, 0], cmap='gray')

#### Note: 

Non-grayscale image should be normalized to [-1,1]

In [6]:
BUFFER_SIZE = 60000
BATCH_SIZE = 256

### Generator network

#### Note:

Modify the network size for deployment

In [7]:
class Generator(Model):

    def __init__(self, noise_dim, image_shape, num_channel):
        super().__init__()
        
        assert len(image_shape) == 2
        assert image_shape[0]%8 == 0
        assert image_shape[1]%8 == 0
        
        self.noise_dim = noise_dim
        self.image_shape = image_shape
        self.num_channel = num_channel
        self.kernel_size = int(min(max(min(image_shape[0]/8.0-3.0, image_shape[1]/8.0-3.0), 3.0), 9.0))

        self.lr_d = layers.ReLU()
        self.lr_c1 = layers.ReLU()
        self.lr_c2 = layers.ReLU()
        self.lr_c3 = layers.ReLU()
        
        self.init_dense = layers.Dense(image_shape[0]/8.0*image_shape[1]/8.0*256,
                               use_bias=False, input_shape=(self.noise_dim,))
        
        self.init_reshape = layers.Reshape((int(image_shape[0]/8.0), int(image_shape[1]/8.0), 256))
        
        self.conv2dT1 = layers.Conv2DTranspose(256, (self.kernel_size, self.kernel_size),
                                               strides=(1, 1), padding='same')
        self.conv2dT2 = layers.Conv2DTranspose(128, (self.kernel_size, self.kernel_size),
                                               strides=(2, 2), padding='same')
        self.conv2dT3 = layers.Conv2DTranspose(64, (self.kernel_size, self.kernel_size),
                                               strides=(2, 2), padding='same')
        self.conv2dTactv = layers.Conv2DTranspose(self.num_channel, (self.kernel_size, self.kernel_size),
                                               strides=(2, 2), padding='same', activation='tanh')

    def call(self, noise_vec):

        init_vec = tf.squeeze(self.lr_d(self.init_dense(noise_vec)))
        
#         print(init_vec.shape)
        
        reshaped = self.init_reshape(init_vec)
        
#         print(reshaped.shape)
        
        convt1 = self.lr_c1(self.conv2dT1(reshaped))

#         print(convt1.shape)
        
        convt2 = self.lr_c2(self.conv2dT2(convt1))
        
#         print(convt2.shape)
                         
        convt3 = self.lr_c3(self.conv2dT3(convt2))
        
#         print(convt3.shape)
            
        out = self.conv2dTactv(convt3)
        
#         print(out.shape)

        return out

#### Testing

In [8]:
g1 = Generator(10, (32, 32), 3)

In [9]:
g1.kernel_size

3

In [10]:
noise_input = tf.random.normal((5, 10))
print(noise_input.shape)
pics1 = g1(tf.expand_dims(noise_input, 0))
print(pics1.shape)
# plt.imshow(pics1[-1, :, :, :], cmap='gray')

(5, 10)
(5, 32, 32, 3)


2022-08-28 10:35:40.592561: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2022-08-28 10:35:40.594448: W tensorflow/stream_executor/gpu/asm_compiler.cc:230] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.6
2022-08-28 10:35:40.594458: W tensorflow/stream_executor/gpu/asm_compiler.cc:233] Used ptxas at ptxas
2022-08-28 10:35:40.594753: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


In [11]:
g1.summary()

Model: "generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 re_lu (ReLU)                multiple                  0         
                                                                 
 re_lu_1 (ReLU)              multiple                  0         
                                                                 
 re_lu_2 (ReLU)              multiple                  0         
                                                                 
 re_lu_3 (ReLU)              multiple                  0         
                                                                 
 dense (Dense)               multiple                  40960     
                                                                 
 reshape (Reshape)           multiple                  0         
                                                                 
 conv2d_transpose (Conv2DTra  multiple                 59

### Discriminator

In [12]:
class Discriminator(Model):

    def __init__(self, image_shape, num_channel):
        super().__init__()
        
        assert len(image_shape) == 2
        assert image_shape[0]%8 == 0
        assert image_shape[1]%8 == 0
        
        self.image_shape = image_shape
        self.num_channel = num_channel
        self.kernel_size = int(min(max(min(image_shape[0]/8.0-3.0, image_shape[1]/8.0-3.0), 3.0), 9.0))

        self.lr_c1 = layers.LeakyReLU()
        self.lr_c2 = layers.LeakyReLU()
        self.lr_c3 = layers.LeakyReLU()
        self.flatten = layers.Flatten()
        
        self.conv2d1 = layers.Conv2D(64, (self.kernel_size, self.kernel_size),
                                        strides=(2, 2), padding='same',
                                        input_shape=(None, self.image_shape[0],
                                        self.image_shape[1], self.num_channel))
        self.conv2d2 = layers.Conv2D(128, (self.kernel_size, self.kernel_size),
                                               strides=(2, 2), padding='same')
        self.conv2d3 = layers.Conv2D(256, (self.kernel_size, self.kernel_size),
                                               strides=(2, 2), padding='same')
        self.dense_actv = layers.Dense(256,
                                      )
#                                        activation="sigmoid")
        
    def call(self, img_input):
        
        conv1 = self.lr_c1(self.conv2d1(img_input))
        
#         print(conv1.shape)
        
        conv2 = self.lr_c2(self.conv2d2(conv1))
        
#         print(conv2.shape)
                         
        conv3 = self.lr_c3(self.conv2d3(conv2))
        
#         print(conv3.shape)
        
        flat = self.flatten(conv3)
        
#         print(flat.shape)
        
        out = tf.squeeze(self.dense_actv(flat))
        
#         print(out.shape)

        return out

#### Testing

In [13]:
d1 = Discriminator((32,32), 3)
g2 = Generator(10, (32,32), 3)
d1.kernel_size

3

In [14]:
noise_input = tf.random.normal((5, 10))
pics2 = g1(tf.expand_dims(noise_input, 0))
# plt.imshow(pics2[-1, :, :, 0], cmap='gray')
print(pics2.shape)

(5, 32, 32, 3)


In [15]:
deci = d1(pics2)
deci

<tf.Tensor: shape=(5, 256), dtype=float32, numpy=
array([[ 7.4471725e-05, -6.6463414e-05,  1.2868243e-04, ...,
         2.4515335e-04,  1.5570488e-04, -3.6045694e-05],
       [ 7.0020978e-05,  1.9221421e-05,  1.1601584e-04, ...,
         9.4312054e-05, -1.5467704e-04, -4.8234925e-04],
       [ 6.8359470e-05,  4.0314870e-04,  4.4166239e-04, ...,
        -1.4375307e-04, -1.9130563e-05, -2.3958864e-04],
       [ 1.2065447e-04,  5.2060367e-04,  7.2034937e-04, ...,
         8.3339590e-05, -2.1140157e-04, -4.8143476e-05],
       [-1.4379679e-05,  5.2060338e-04,  9.6209795e-04, ...,
         2.5674430e-04, -7.7900680e-05, -4.2041403e-04]], dtype=float32)>

In [16]:
d1.summary()

Model: "discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 leaky_re_lu (LeakyReLU)     multiple                  0         
                                                                 
 leaky_re_lu_1 (LeakyReLU)   multiple                  0         
                                                                 
 leaky_re_lu_2 (LeakyReLU)   multiple                  0         
                                                                 
 flatten (Flatten)           multiple                  0         
                                                                 
 conv2d (Conv2D)             multiple                  1792      
                                                                 
 conv2d_1 (Conv2D)           multiple                  73856     
                                                                 
 conv2d_2 (Conv2D)           multiple                

### CramerDCGAN

In [17]:
EPSILON = 1e-16

class DCGAN:
    
    def __init__(self, dataset, image_shape, num_channel, noise_latent_dim,
                 buffer_size=60000, batch_size=256, lr=3e-4, gp_lam = 10.0):
        assert len(image_shape) == 2
        assert image_shape[0]%8 == 0
        assert image_shape[1]%8 == 0
        
        self.image_shape = image_shape
        self.num_channel = num_channel
        self.noise_latent_dim = noise_latent_dim
        self.buffer_cap, self.batch_size, self.gp_lam = buffer_size, batch_size, gp_lam
        self.num_img_prog_monit = 16
        
        self.dataset = dataset
        # NOTE: Dataset must be processed differently for different source and applications
        
        self.g = Generator(self.noise_latent_dim, self.image_shape, self.num_channel)
        self.d = Discriminator(self.image_shape, self.num_channel)
        
        self.g_opt = tf.keras.optimizers.Adam(lr)
        self.d_opt = tf.keras.optimizers.Adam(lr)
        
        self.g_seed = tf.random.normal((self.num_img_prog_monit, self.noise_latent_dim))

    def cramer_loss(self, d_x_data, d_g_z_1, d_g_z_2, x_it):
        
        crit_r = tf.math.add(tf.math.sqrt(tf.reduce_sum(tf.math.add(d_x_data, -d_g_z_2)**2, axis = 1)+EPSILON),
                   -tf.math.sqrt(tf.reduce_sum(d_x_data**2, axis = 1)+EPSILON))
        crit_g_1 = tf.math.add(tf.math.sqrt(tf.reduce_sum(tf.math.add(d_g_z_1, -d_g_z_2)**2, axis = 1)+EPSILON),
                   -tf.math.sqrt(tf.reduce_sum(d_g_z_1**2, axis = 1)+EPSILON))
        
        L_srg = tf.math.add(crit_r, -crit_g_1)
        
        with tf.GradientTape() as t_gp:
            t_gp.watch(x_it)
            d_it = self.d(x_it)
            crit_it = tf.math.add(tf.math.sqrt(tf.reduce_sum(tf.math.add(d_it, -d_g_z_2)**2, axis = 1)+EPSILON),
                   -tf.math.sqrt(tf.reduce_sum(d_it**2, axis = 1)+EPSILON))
            
        gp_grad = t_gp.gradient(crit_it, x_it)
        l2n_gp = tf.math.sqrt(tf.reduce_sum(gp_grad**2, axis = [1,2,3])+EPSILON)
        
        # d_loss
        L_d = tf.reduce_mean(-L_srg + (self.gp_lam*((l2n_gp-1.0)**2)))

        l2nrg1 = tf.math.sqrt(tf.reduce_sum(tf.math.add(d_x_data, -d_g_z_1)**2, axis = 1)+EPSILON)
        l2nrg2 = tf.math.sqrt(tf.reduce_sum(tf.math.add(d_x_data, -d_g_z_2)**2, axis = 1)+EPSILON)
        l2ng12 = tf.math.sqrt(tf.reduce_sum(tf.math.add(d_g_z_1, -d_g_z_2)**2, axis = 1)+EPSILON)

        # g_loss
        L_g = tf.reduce_mean(l2nrg1 + l2nrg2 - l2ng12)

        return L_g, L_d
        
    
    @tf.function
    def update(self, imgs):
        noise_input1 = tf.random.normal((imgs.shape[0], self.noise_latent_dim))
        noise_input2 = tf.random.normal((imgs.shape[0], self.noise_latent_dim))
        
        with tf.GradientTape() as g_tape, tf.GradientTape() as d_tape:
            g_z_1 = self.g(noise_input1)
            g_z_2 = self.g(noise_input2)
            
            d_x_data = self.d(imgs)
            d_g_z_1 = self.d(g_z_1)
            d_g_z_2 = self.d(g_z_2)
            
            epsi = tf.random.uniform([imgs.shape[0], 1, 1, 1], 0.0, 1.0)
            x_it = tf.math.add(epsi*imgs, (1.0-epsi)*g_z_1)
            g_loss, d_loss = self.cramer_loss(d_x_data, d_g_z_1, d_g_z_2, x_it)
            
        grad_g = g_tape.gradient(g_loss, self.g.trainable_variables)
        grad_d = d_tape.gradient(d_loss, self.d.trainable_variables)
        
        self.g_opt.apply_gradients(zip(grad_g, self.g.trainable_variables))
        self.d_opt.apply_gradients(zip(grad_d, self.d.trainable_variables))
        return g_loss, d_loss
        
    def train(self, epochs=50):
        for epo in range(epochs):
            for img_b, l_b in self.dataset:
                norm_img_b = (img_b-127.5)/127.5
                g_l, d_l = self.update(norm_img_b)
            print("Generator Loss: ", g_l, ", Discriminator Loss: ",  d_l)
            
            self.monitor_progress(epo)
            
    def monitor_progress(self, epo):
        pics = self.g(self.g_seed)
        
        fig = plt.figure(figsize=(4,4))
        for i in range(pics.shape[0]):
            plt.subplot(4,4,i+1)
            plt.imshow(tf.cast(tf.math.round(pics[i,:,:,:]*127.5+127.5), tf.int32))
            plt.axis('off')
            
#         plt.savefig('/Users/anthonylaw/Desktop/Hustle/GAN-devel/imgs/image_{:04d}.png'.format(epo))
        plt.savefig('/home/tony/TO_BE_REMOVED/imgs/image_{:04d}.png'.format(epo))
        # NEEDS to be changed for machines
        
        plt.close('all')
        

#### Testing

In [18]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  "/home/tony/TO_BE_REMOVED/celeba_data/imgs/",
  seed=123,
  image_size=(64, 64),
  batch_size=256)

Found 202599 files belonging to 1 classes.


In [19]:
dcgan1 = DCGAN(train_ds, (64, 64), 3, 100)

In [ ]:
dcgan1.train(1000)